<a href="https://colab.research.google.com/github/zooodung/TA-Prediction/blob/main/Preprocess_LabelEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교통사고 위험도 예측

## 라이브러리 및 파일 호출

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytimekr import pytimekr
from sklearn.preprocessing import LabelEncoder
import os

In [5]:
path=os.getcwd()

In [6]:
daegu_data=pd.read_csv(path + f'/데이터들/전처리 전/train.csv')
wide_data=pd.read_csv(path + f'/데이터들/전처리 전/external_open/countrywide_accident.zip')
test_data=pd.read_csv(path + f'/데이터들/전처리 전/test.csv')

## 기초 통계 분석

In [7]:
#ta_data_daegu.info()

In [8]:
#x_test.info()

In [9]:
#ta_data_daegu.describe()

In [10]:
#ta_data_daegu.describe()

In [11]:
#columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형']

#for column in columns:
#    print(f"Column: {column}")
#    print(ta_data_daegu[column].value_counts())
#    print("-" * 30)

## 전처리

### 대구 데이터 전처리

#### 불필요한 컬럼 제거

In [12]:
columns_to_drop = ['ID', '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령',
                   '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도']


daegu=daegu_data.drop(columns=columns_to_drop)

#### 사고일시 컬럼 연도, 월, 일, 시간 단위로 분할

In [13]:
#daegu[['년도','월','일+시간']] = daegu['사고일시'].str.split('-',expand=True)
#daegu[['일','시간']]= daegu['일+시간'].str.split(' ',expand=True)

#daegu.drop(['일+시간','사고일시'],inplace=True, axis=1)
#daegu.drop(['일'],inplace=True, axis=1)

#### 시군구 단위로 분할

In [14]:
daegu[['시','구','동']] = daegu['시군구'].str.split(' ',expand=True)
daegu.drop(['시군구'],inplace=True, axis=1)

#### 도로 형태 분할

In [15]:
daegu[['도로형태1', '도로형태2']] = daegu['도로형태'].str.split(' - ', expand=True)
daegu.drop(['도로형태'],inplace=True, axis=1)

In [16]:
daegu.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타


#### 요일 컬럼 평일과 주말로 분류

In [17]:
#ta_data_daegu['요일 분류'] = ta_data_daegu['요일'].apply(lambda x: '평일' if x in ['월요일', '화요일', '수요일', '목요일', '금요일'] else '주말')

In [18]:
#ta_data_daegu

### 전국 데이터 전처리

In [19]:
wide=wide_data.copy()
len(wide)

602775

#### 철길 건널목 제거

In [20]:
wide = wide[~wide['도로형태'].str.split().str[-1].str.contains('철길건널목')]

#### 불필요한 컬럼 제거

In [21]:
wide=wide.drop(columns=columns_to_drop)

#### 광역시, 특별시만 분류

In [22]:
def contains_specific_location(address):
    cities = ['서울특별시', '인천광역시', '부산광역시', '울산광역시', '광주광역시', '대전광역시']
    for city in cities:
        if city in address:
            return True
    return False
wide=wide[wide['시군구'].apply(contains_specific_location)]

wide[['시','구','동']] = wide['시군구'].str.split(' ',expand=True)

In [23]:
wide['시군구'].apply(lambda x:x.split()[0]).value_counts()

시군구
서울특별시    108395
부산광역시     36244
인천광역시     25115
광주광역시     23430
대전광역시     22579
울산광역시     11981
Name: count, dtype: int64

In [24]:
wide.drop(['시군구'],inplace=True, axis=1)

#### 연도, 월, 일, 시간 분류

In [25]:
#wide[['년도','월','일+시간']] = wide['사고일시'].str.split('-',expand=True)
#wide[['일','시간']]= wide['일+시간'].str.split(' ',expand=True)

#wide.drop(['일+시간','사고일시'],inplace=True, axis=1)
#wide.drop(['일'],inplace=True, axis=1)

#### 도로 형태 분류

In [26]:
wide[['도로형태1', '도로형태2']] = wide['도로형태'].str.split(' - ', expand=True)

wide.drop(['도로형태'],inplace=True, axis=1)

In [27]:
wide.head()

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,강서구,방화동,교차로,교차로횡단보도내
4,2019-01-01 00,화요일,맑음,건조,차대차,0,0,2,1,7,인천광역시,부평구,부평동,교차로,교차로안
5,2019-01-01 00,화요일,맑음,건조,차대사람,0,0,1,0,3,인천광역시,부평구,부평동,교차로,교차로부근
8,2019-01-01 00,화요일,맑음,건조,차대차,0,0,1,0,3,광주광역시,광산구,수완동,단일로,기타
11,2019-01-01 00,화요일,맑음,건조,차대사람,1,0,0,0,10,부산광역시,해운대구,우동,교차로,교차로안


### 데이터 병합 후 전처리 진행

In [28]:
print(len(daegu),len(wide), len(daegu)+len(wide))

39609 227744 267353


In [29]:
concatenated_df = pd.concat([daegu, wide])
concatenated_df

,사고일시,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2
0,2019-01-01 00,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타
1,2019-01-01 00,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타
2,2019-01-01 01,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타
3,2019-01-01 02,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타
4,2019-01-01 04,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,2021-12-31 23,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타
602769,2021-12-31 23,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타
602770,2021-12-31 23,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안
602771,2021-12-31 23,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타


#### 휴일 여부

In [30]:
for i in range(2019, 2024):
    kr_holidays = pytimekr.holidays(i)
    print(kr_holidays)

[datetime.date(2019, 2, 4), datetime.date(2019, 2, 5), datetime.date(2019, 2, 6), datetime.date(2019, 9, 12), datetime.date(2019, 9, 13), datetime.date(2019, 9, 14), datetime.date(2019, 1, 1), datetime.date(2019, 3, 1), datetime.date(2019, 5, 5), datetime.date(2019, 5, 12), datetime.date(2019, 6, 6), datetime.date(2019, 8, 15), datetime.date(2019, 10, 9), datetime.date(2019, 10, 3), datetime.date(2019, 12, 25)]
[datetime.date(2020, 1, 24), datetime.date(2020, 1, 25), datetime.date(2020, 1, 26), datetime.date(2020, 9, 30), datetime.date(2020, 10, 1), datetime.date(2020, 10, 2), datetime.date(2020, 1, 1), datetime.date(2020, 3, 1), datetime.date(2020, 5, 5), datetime.date(2020, 4, 30), datetime.date(2020, 6, 6), datetime.date(2020, 8, 15), datetime.date(2020, 10, 9), datetime.date(2020, 10, 3), datetime.date(2020, 12, 25)]
[datetime.date(2021, 2, 11), datetime.date(2021, 2, 12), datetime.date(2021, 2, 13), datetime.date(2021, 9, 20), datetime.date(2021, 9, 21), datetime.date(2021, 9, 22)

In [31]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [32]:
concatenated_df['사고일시'] = pd.to_datetime(concatenated_df['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
concatenated_df['년도'] = concatenated_df['사고일시'].dt.year
concatenated_df['월'] = concatenated_df['사고일시'].dt.month
concatenated_df['day'] = concatenated_df['사고일시'].dt.day
concatenated_df['시간'] = concatenated_df['사고일시'].dt.hour
concatenated_df['day_of_week'] = concatenated_df['사고일시'].dt.dayofweek

In [33]:
concatenated_df['holiday'] = np.where((concatenated_df.day_of_week >= 5) | (concatenated_df.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

In [34]:
concatenated_df.drop(columns=['사고일시'], inplace=True)

In [35]:
concatenated_df['holiday'].value_counts()

holiday
평일    195393
휴일     71960
Name: count, dtype: int64

In [36]:
concatenated_df.drop(['day', 'day_of_week'], inplace=True, axis=1)

#### 주말 여부

In [37]:
#concatenated_df['주말여부']=concatenated_df['요일'].apply(lambda x:'주말' if x in ['토요일','일요일'] else '평일')

#concatenated_df['주말여부'].value_counts()

#### 시간 분류

In [38]:
def seperate_time(time): #심야, 새벽, 오전, 오후, 저녁, 야간 ->4시간단위
    if 0<=int(time)<4:
        return '심야'
    elif int(time)<8:
        return '새벽'
    elif int(time)<12:
        return '오전'
    elif int(time)<16:
        return '오후'
    elif int(time)<20:
        return '저녁'
    else:
        return '야간'
concatenated_df['시간구분']=concatenated_df['시간'].apply(seperate_time)
print(concatenated_df['시간구분'].value_counts().sum())
concatenated_df['시간구분'].value_counts()

267353


시간구분
저녁    71768
오후    58743
오전    50058
야간    47737
새벽    20554
심야    18493
Name: count, dtype: int64

#### 계절 분류

In [39]:
def seperate_month(month):
    if int(month) in [12,1,2]:
        return '겨울'
    elif int(month) in [3,4,5]:
        return '봄'
    elif int(month) in [6,7,8]:
        return '여름'
    else:
        return '가을'
concatenated_df['계절']=concatenated_df['월'].apply(seperate_month)
print(concatenated_df['계절'].value_counts().sum())
concatenated_df['계절'].value_counts()

267353


계절
가을    70234
여름    68434
봄     66607
겨울    62078
Name: count, dtype: int64

### 병합된 데이터 전처리 확인

In [40]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,중구,대신동,단일로,기타,2019,1,0,휴일,심야,겨울
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,달서구,감삼동,단일로,기타,2019,1,0,휴일,심야,겨울
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,수성구,두산동,단일로,기타,2019,1,1,휴일,심야,겨울
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,북구,복현동,단일로,기타,2019,1,2,휴일,심야,겨울
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,동구,신암동,단일로,기타,2019,1,4,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,서초구,반포동,단일로,기타,2021,12,23,평일,야간,겨울
602769,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,연제구,연산동,단일로,기타,2021,12,23,평일,야간,겨울
602770,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,해운대구,우동,교차로,교차로안,2021,12,23,평일,야간,겨울
602771,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,성동구,행당동,단일로,기타,2021,12,23,평일,야간,겨울


### 테스트 데이터 전처리 진행

In [41]:
test=test_data.copy()

In [42]:
test[['시','구','동']] = test['시군구'].str.split(' ',expand=True)
test.drop(['시군구'],inplace=True, axis=1)

In [43]:
test[['도로형태1', '도로형태2']] = test['도로형태'].str.split(' - ', expand=True)
test.drop(['도로형태'],inplace=True, axis=1)

In [44]:
kr_holidays = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-05', '2023-05-26', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25']


In [45]:
test['사고일시'] = pd.to_datetime(test['사고일시'])

# datetime에서 year, month, day, hour 추출하여 새로운 열 생성
test['년도'] = test['사고일시'].dt.year
test['월'] = test['사고일시'].dt.month
test['day'] = test['사고일시'].dt.day
test['시간'] = test['사고일시'].dt.hour
test['day_of_week'] = test['사고일시'].dt.dayofweek

test['holiday'] = np.where((test.day_of_week >= 5) | (test.사고일시.dt.strftime('%Y-%m-%d').isin(kr_holidays)), '휴일', '평일')

test.drop(columns=['사고일시'], inplace=True)
test.drop(['day', 'day_of_week'], inplace=True, axis=1)

In [46]:
test['시간구분']=test['시간'].apply(seperate_time)

In [47]:
test['계절']=test['월'].apply(seperate_month)
test['계절'].value_counts()

계절
가을    2947
여름    2783
봄     2771
겨울    2462
Name: count, dtype: int64

#### 테스트 데이터 전처리 결과 확인

In [48]:
test

,ID,요일,기상상태,노면상태,사고유형,시,구,동,도로형태1,도로형태2,년도,월,시간,holiday,시간구분,계절
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,대구광역시,수성구,상동,교차로,교차로안,2022,1,1,휴일,심야,겨울
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,대구광역시,수성구,지산동,단일로,기타,2022,1,1,휴일,심야,겨울
2,ACCIDENT_39611,토요일,맑음,건조,차대차,대구광역시,수성구,수성동2가,교차로,교차로안,2022,1,4,휴일,새벽,겨울
3,ACCIDENT_39612,토요일,맑음,건조,차대차,대구광역시,수성구,신매동,단일로,기타,2022,1,4,휴일,새벽,겨울
4,ACCIDENT_39613,토요일,맑음,건조,차대차,대구광역시,달서구,감삼동,교차로,교차로안,2022,1,6,휴일,새벽,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,토요일,맑음,건조,차대차,대구광역시,남구,대명동,단일로,터널,2022,12,18,휴일,저녁,겨울
10959,ACCIDENT_50568,토요일,맑음,건조,차대차,대구광역시,수성구,시지동,단일로,기타,2022,12,18,휴일,저녁,겨울
10960,ACCIDENT_50569,토요일,맑음,건조,차대차,대구광역시,수성구,연호동,단일로,기타,2022,12,20,휴일,야간,겨울
10961,ACCIDENT_50570,토요일,맑음,건조,차대차,대구광역시,수성구,범물동,교차로,교차로부근,2022,12,20,휴일,야간,겨울


In [49]:
concatenated_df = concatenated_df[~concatenated_df['기상상태'].str.split().str[-1].str.contains('안개')]
concatenated_df = concatenated_df[~concatenated_df['노면상태'].str.split().str[-1].str.contains('해빙')]

### 수치형 데이터로 변환 (레이블 인코딩)

#### 레이블 인코더 생성

In [50]:
label_encoder = LabelEncoder()

#### 기상상태 인코딩

In [51]:
concatenated_df['기상상태'].value_counts()

기상상태
맑음    238065
비      18056
흐림      8224
기타      2209
눈        746
Name: count, dtype: int64

In [52]:
concatenated_df['기상상태_레이블'] = label_encoder.fit_transform(concatenated_df['기상상태'])
concatenated_df['기상상태_레이블'].value_counts()

기상상태_레이블
2    238065
3     18056
4      8224
0      2209
1       746
Name: count, dtype: int64

#### 노면상태 변환

In [53]:
concatenated_df['노면상태'].value_counts()

노면상태
건조       241306
젖음/습기     22766
기타         2323
서리/결빙       555
적설          325
침수           25
Name: count, dtype: int64

In [54]:
concatenated_df['노면상태_레이블'] = label_encoder.fit_transform(concatenated_df['노면상태'])
concatenated_df['노면상태_레이블'].value_counts()

노면상태_레이블
0    241306
4     22766
1      2323
2       555
3       325
5        25
Name: count, dtype: int64

#### 사고유형 변환

In [55]:
concatenated_df['사고유형'].value_counts()

사고유형
차대차     203978
차대사람     55386
차량단독      7936
Name: count, dtype: int64

In [56]:
concatenated_df['사고유형_레이블'] = label_encoder.fit_transform(concatenated_df['사고유형'])
concatenated_df['사고유형_레이블'].value_counts()

사고유형_레이블
1    203978
0     55386
2      7936
Name: count, dtype: int64

#### 시 변환

In [57]:
concatenated_df['시'].value_counts()

시
서울특별시    108384
대구광역시     39601
부산광역시     36241
인천광역시     25102
광주광역시     23423
대전광역시     22570
울산광역시     11979
Name: count, dtype: int64

In [58]:
concatenated_df['시_레이블'] = label_encoder.fit_transform(concatenated_df['시'])
concatenated_df['시_레이블'].value_counts()

시_레이블
4    108384
1     39601
3     36241
6     25102
0     23423
2     22570
5     11979
Name: count, dtype: int64

#### 구 변환

In [59]:
concatenated_df['구'].value_counts()

구
서구      23097
북구      17558
동구      14266
중구      13169
강남구     11293
남구      11291
달서구      9890
송파구      8093
강서구      7085
영등포구     6824
서초구      6785
수성구      6706
광산구      6415
유성구      5730
남동구      5313
부산진구     4894
노원구      4577
중랑구      4440
동대문구     4401
관악구      4226
구로구      4203
양천구      4154
부평구      4154
성북구      4069
마포구      3868
동작구      3859
강동구      3728
미추홀구     3520
강북구      3381
성동구      3267
용산구      3091
서대문구     2970
사하구      2954
해운대구     2946
은평구      2938
동래구      2935
종로구      2814
달성군      2765
광진구      2764
사상구      2726
금천구      2557
계양구      2528
금정구      2491
대덕구      2469
울주군      2373
연제구      2278
도봉구      2184
수영구      2121
연수구      1865
기장군      1575
영도구      1021
강화군       580
옹진군        99
Name: count, dtype: int64

In [60]:
concatenated_df['구_레이블'] = label_encoder.fit_transform(concatenated_df['구'])
concatenated_df['구_레이블'].value_counts()

구_레이블
31    23097
28    17558
20    14266
50    13169
0     11293
13    11291
16     9890
36     8093
3      7085
43     6824
33     6785
37     6706
7      6415
47     5730
14     5313
26     4894
15     4577
51     4440
21     4401
6      4226
9      4203
39     4154
27     4154
35     4069
24     3868
23     3859
1      3728
25     3520
2      3381
34     3267
45     3091
32     2970
30     2954
52     2946
48     2938
22     2935
49     2814
17     2765
8      2764
29     2726
11     2557
5      2528
10     2491
18     2469
46     2373
41     2278
19     2184
38     2121
40     1865
12     1575
42     1021
4       580
44       99
Name: count, dtype: int64

#### 동 변환

In [61]:
concatenated_df['동'].value_counts()

동
논현동      2518
역삼동      2427
신정동      2400
신림동      2268
서초동      1963
         ... 
수하동         1
우명동         1
회현동3가       1
항동4가        1
덕림동         1
Name: count, Length: 1283, dtype: int64

In [62]:
concatenated_df['동_레이블'] = label_encoder.fit_transform(concatenated_df['동'])
concatenated_df['동_레이블'].value_counts()

동_레이블
194     2518
806     2427
739     2400
714     2268
617     1963
        ... 
689        1
901        1
1268       1
1222       1
264        1
Name: count, Length: 1283, dtype: int64

#### 도로형태1 변환

In [63]:
concatenated_df['도로형태1'].value_counts()

도로형태1
단일로    130297
교차로    123217
기타      12647
주차장      1047
미분류        92
Name: count, dtype: int64

In [64]:
concatenated_df['도로형태1_레이블'] = label_encoder.fit_transform(concatenated_df['도로형태1'])
concatenated_df['도로형태1_레이블'].value_counts()

도로형태1_레이블
2    130297
0    123217
1     12647
4      1047
3        92
Name: count, dtype: int64

#### 도로형태2 변환

In [65]:
concatenated_df['도로형태2'].value_counts()

도로형태2
기타           136027
교차로안          73484
교차로부근         39998
교차로횡단보도내       9735
지하차도(도로)내      2857
교량             2092
고가도로위          1114
주차장            1047
터널              854
미분류              92
Name: count, dtype: int64

In [66]:
concatenated_df['도로형태2_레이블'] = label_encoder.fit_transform(concatenated_df['도로형태2'])
concatenated_df['도로형태2_레이블'].value_counts()

도로형태2_레이블
5    136027
3     73484
2     39998
4      9735
8      2857
1      2092
0      1114
7      1047
9       854
6        92
Name: count, dtype: int64

#### holiday 변환

In [67]:
concatenated_df['holiday'].value_counts()

holiday
평일    195354
휴일     71946
Name: count, dtype: int64

In [68]:
concatenated_df['holiday_레이블'] = label_encoder.fit_transform(concatenated_df['holiday'])
concatenated_df['holiday_레이블'].value_counts()

holiday_레이블
0    195354
1     71946
Name: count, dtype: int64

#### 시간 변환

In [69]:
concatenated_df['시간구분'].value_counts()

시간구분
저녁    71767
오후    58741
오전    50045
야간    47732
새벽    20533
심야    18482
Name: count, dtype: int64

In [70]:
concatenated_df['시간구분_레이블'] = label_encoder.fit_transform(concatenated_df['시간구분'])
concatenated_df['시간구분_레이블'].value_counts()

시간구분_레이블
5    71767
4    58741
3    50045
2    47732
0    20533
1    18482
Name: count, dtype: int64

#### 계절 변환

In [71]:
concatenated_df['계절'].value_counts()

계절
가을    70225
여름    68428
봄     66593
겨울    62054
Name: count, dtype: int64

In [72]:
concatenated_df['계절_레이블'] = label_encoder.fit_transform(concatenated_df['계절'])
concatenated_df['계절_레이블'].value_counts()

계절_레이블
0    70225
3    68428
2    66593
1    62054
Name: count, dtype: int64

### 테스트 데이터 레이블 인코딩

In [73]:
#기상상태
test['기상상태_레이블'] = label_encoder.fit_transform(test['기상상태'])
#노면상태
test['노면상태_레이블'] = label_encoder.fit_transform(test['노면상태'])
#사고유형
test['사고유형_레이블'] = label_encoder.fit_transform(test['사고유형'])
#시
test['시_레이블'] = label_encoder.fit_transform(test['시'])
#구
test['구_레이블'] = label_encoder.fit_transform(test['구'])
#동
test['동_레이블'] = label_encoder.fit_transform(test['동'])
#도로형태1
test['도로형태1_레이블'] = label_encoder.fit_transform(test['도로형태1'])
#도로형태2
test['도로형태2_레이블'] = label_encoder.fit_transform(test['도로형태2'])
#holiday
test['holiday_레이블'] = label_encoder.fit_transform(test['holiday'])
#시간
test['시간구분_레이블'] = label_encoder.fit_transform(test['시간구분'])
#계절
test['계절_레이블'] = label_encoder.fit_transform(test['계절'])

### 레이블 인코딩 결과 확인

In [74]:
concatenated_df

,요일,기상상태,노면상태,사고유형,사망자수,중상자수,경상자수,부상자수,ECLO,시,...,노면상태_레이블,사고유형_레이블,시_레이블,구_레이블,동_레이블,도로형태1_레이블,도로형태2_레이블,holiday_레이블,시간구분_레이블,계절_레이블
0,화요일,맑음,건조,차대사람,0,1,0,0,5,대구광역시,...,0,0,1,50,236,2,5,1,1,1
1,화요일,흐림,건조,차대사람,0,0,1,0,3,대구광역시,...,0,0,1,16,23,2,5,1,1,1
2,화요일,맑음,건조,차대사람,0,0,1,0,3,대구광역시,...,0,0,1,37,343,2,5,1,1,1
3,화요일,맑음,건조,차대차,0,1,0,0,5,대구광역시,...,0,1,1,28,476,2,5,1,1,1
4,화요일,맑음,건조,차대차,0,0,1,0,3,대구광역시,...,0,1,1,20,732,2,5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602766,금요일,맑음,건조,차대사람,0,0,1,0,3,서울특별시,...,0,0,4,33,432,2,5,0,2,1
602769,금요일,맑음,건조,차대차,0,1,0,0,5,부산광역시,...,0,1,3,41,811,2,5,0,2,1
602770,금요일,흐림,건조,차대차,0,1,1,0,8,부산광역시,...,0,1,3,52,899,0,3,0,2,1
602771,금요일,맑음,건조,차대차,0,0,2,0,6,서울특별시,...,0,1,4,34,1228,2,5,0,2,1


In [75]:
test

,ID,요일,기상상태,노면상태,사고유형,시,구,동,도로형태1,도로형태2,...,노면상태_레이블,사고유형_레이블,시_레이블,구_레이블,동_레이블,도로형태1_레이블,도로형태2_레이블,holiday_레이블,시간구분_레이블,계절_레이블
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,대구광역시,수성구,상동,교차로,교차로안,...,0,0,0,6,96,0,3,1,1,1
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,대구광역시,수성구,지산동,단일로,기타,...,0,0,0,6,162,2,5,1,1,1
2,ACCIDENT_39611,토요일,맑음,건조,차대차,대구광역시,수성구,수성동2가,교차로,교차로안,...,0,1,0,6,113,0,3,1,0,1
3,ACCIDENT_39612,토요일,맑음,건조,차대차,대구광역시,수성구,신매동,단일로,기타,...,0,1,0,6,122,2,5,1,0,1
4,ACCIDENT_39613,토요일,맑음,건조,차대차,대구광역시,달서구,감삼동,교차로,교차로안,...,0,1,0,1,4,0,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,토요일,맑음,건조,차대차,대구광역시,남구,대명동,단일로,터널,...,0,1,0,0,37,2,9,1,5,1
10959,ACCIDENT_50568,토요일,맑음,건조,차대차,대구광역시,수성구,시지동,단일로,기타,...,0,1,0,6,119,2,5,1,5,1
10960,ACCIDENT_50569,토요일,맑음,건조,차대차,대구광역시,수성구,연호동,단일로,기타,...,0,1,0,6,129,2,5,1,2,1
10961,ACCIDENT_50570,토요일,맑음,건조,차대차,대구광역시,수성구,범물동,교차로,교차로부근,...,0,1,0,6,74,0,2,1,2,1


In [ ]:
concatenated_df.to_csv(path + f'/데이터들/라벨 인코딩/train.csv')
test.to_csv(path + f'/데이터들/라벨 인코딩/test.csv')